# Usando nuevo dataset

# Importando dataset

In [414]:
import jsonlines
import pandas as pd

with jsonlines.open('Dataset/SMHD/SMHD_eating_train.jsonl') as reader:
    count=0
    df_train = pd.DataFrame()
    for obj in reader:
        count=count+1
        lines = reader.read()
        df_inter=pd.json_normalize(lines, record_path=['posts'], meta=['id', 'label'])
        df_train=pd.concat([df_train,df_inter], axis=0)
        
with jsonlines.open('Dataset/SMHD/SMHD_eating_test.jsonl') as reader:
    count=0
    df_test = pd.DataFrame()
    for obj in reader:
        count=count+1
        lines = reader.read()
        df_inter=pd.json_normalize(lines, record_path=['posts'], meta=['id', 'label'])
        df_test=pd.concat([df_test,df_inter], axis=0)

### Borramos columnas innecesarias

In [415]:
df_train.drop(['selftext', 'body', 'created_utc', 'title'], axis=1, inplace=True)
df_test.drop(['selftext', 'body', 'created_utc', 'title'], axis=1, inplace=True)

# Hacer join de los text de los mismos ids

In [433]:
df_train_grouped=df_train.groupby(['id', 'label']).text.apply(list).transform(lambda x : ' '.join(x)).reset_index()
df_test_grouped=df_test.groupby(['id', 'label']).text.apply(list).transform(lambda x : ' '.join(x)).reset_index()

In [434]:
df_test_grouped

,id,label,text
0,1067975,0,Nice picture of it while she was still intact...
1,1129224,1,"It's odd to me, because I had the exact opposi..."
2,1133773,0,"No. It's egg yolk, oil, and vinegar (or citrus..."
3,1214015,1,The island looks like a pizza. 1988 Volkswagen...
4,1292839,1,Just finished my major post grad project. Took...
...,...,...,...
107,9763340,1,I had the severe misfortune to stumble across ...
108,9801892,0,Being more present after school in the hallway...
109,9803370,1,[removed] Track - stay under\nWater - more th...
110,9839533,1,Probably be good on r/bdsm too. I've always f...


# Tokenizado y eliminacion de stopwords

In [435]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

def preprocess(words, type='doc'):
    tokens = nltk.word_tokenize(words)
    porter = nltk.PorterStemmer()
    lemmas = [porter.stem(t) for t in tokens]
    stoplist = stopwords.words('english')
    lemmas_clean = [w for w in tokens if w not in stoplist]
    punctuation = set(string.punctuation)
    words = [w for w in lemmas_clean if  w not in punctuation]
    return words

def preprocesssimple(words, type='doc'):
    stoplist = stopwords.words('english')
    lemmas_clean = [w for w in words if w not in stoplist]
    return words

In [436]:
#df_train_grouped['text'] = df_train_grouped.apply(lambda row: preprocess(row['text']), axis=1)
#df_test_grouped['text'] = df_test_grouped.apply(lambda row: preprocess(row['text']), axis=1)
df_train_grouped['text'] = df_train_grouped.apply(lambda row: preprocesssimple(row['text']), axis=1)
df_test_grouped['text'] = df_test_grouped.apply(lambda row: preprocesssimple(row['text']), axis=1)

In [437]:
df_train_grouped

,id,label,text
0,1009120,1,A decent salad at a non-chain restaurant. No d...
1,1017198,0,"disagree. blue shirt. :) this is adorable, s..."
2,1221243,1,Why not call it r/BestOfTwitter? I could be ...
3,1288853,1,I haven't begun my career quite yet after comp...
4,1328156,0,"""................................................"
...,...,...,...
99,9555467,0,I thought this referred to Salk? I only rememb...
100,9621473,1,"You will know by the next day, your muscles wi..."
101,9637680,1,I gasped so loud at that first sentence that I...
102,9684845,1,I put everything off until the very last minut...


In [438]:
df_test_grouped

,id,label,text
0,1067975,0,Nice picture of it while she was still intact...
1,1129224,1,"It's odd to me, because I had the exact opposi..."
2,1133773,0,"No. It's egg yolk, oil, and vinegar (or citrus..."
3,1214015,1,The island looks like a pizza. 1988 Volkswagen...
4,1292839,1,Just finished my major post grad project. Took...
...,...,...,...
107,9763340,1,I had the severe misfortune to stumble across ...
108,9801892,0,Being more present after school in the hallway...
109,9803370,1,[removed] Track - stay under\nWater - more th...
110,9839533,1,Probably be good on r/bdsm too. I've always f...


# Modelizacion

In [439]:
import sklearn
from sklearn.model_selection import train_test_split

x_train = df_train_grouped['text']
y_train = df_train_grouped['label']
x_test = df_test_grouped['text']
y_test = df_test_grouped['label']
#x = df_test_grouped['text']
#y = df_test_grouped['label']

#vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')
vectorizer=CountVectorizer()

x_train = vectorizer.fit_transform(x_train.apply(lambda x: ' '.join(x)))
x_test = vectorizer.fit_transform(x_test.apply(lambda x: ' '.join(x)))
#x = vectorizer.fit_transform(x.apply(lambda x: ' '.join(x)))

#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [423]:
x_train

<104x32090 sparse matrix of type '<class 'numpy.float64'>'
	with 136016 stored elements in Compressed Sparse Row format>

In [424]:
x_test

<112x36527 sparse matrix of type '<class 'numpy.float64'>'
	with 165459 stored elements in Compressed Sparse Row format>

In [384]:
df_train_grouped

,id,label,text
0,1009120,1,A decent salad at a non-chain restaurant. No d...
1,1017198,0,"disagree. blue shirt. :) this is adorable, s..."
2,1221243,1,Why not call it r/BestOfTwitter? I could be ...
3,1288853,1,I haven't begun my career quite yet after comp...
4,1328156,0,"""................................................"
...,...,...,...
99,9555467,0,I thought this referred to Salk? I only rememb...
100,9621473,1,"You will know by the next day, your muscles wi..."
101,9637680,1,I gasped so loud at that first sentence that I...
102,9684845,1,I put everything off until the very last minut...


In [385]:
df_test_grouped

,id,label,text
0,1067975,0,Nice picture of it while she was still intact...
1,1129224,1,"It's odd to me, because I had the exact opposi..."
2,1133773,0,"No. It's egg yolk, oil, and vinegar (or citrus..."
3,1214015,1,The island looks like a pizza. 1988 Volkswagen...
4,1292839,1,Just finished my major post grad project. Took...
...,...,...,...
107,9763340,1,I had the severe misfortune to stumble across ...
108,9801892,0,Being more present after school in the hallway...
109,9803370,1,[removed] Track - stay under\nWater - more th...
110,9839533,1,Probably be good on r/bdsm too. I've always f...


In [386]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer

classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
y_train=y_train.astype('int')
classifier.fit(x_train, y_train)

ValueError: could not convert string to float: 'A decent salad at a non-chain restaurant. No dessert, though. three horse head emojis in a row. I had a friend [41F] who went through a bad breakup. After two years, she was still *very* upset about it. Since we both went through bad breakups at the same time (mine from a physically, verbally, and emotionally abusive man who I lived with for three years), we were very supportive of one another initially, and even moved in together as roommates - but I got over my breakup and she did not. She would want to talk about her ex and what had happened with them for hours and hours at a time, whenever she wanted to do it, and since we lived together I had little escape from listening to her re-hash the situation and talk about how painful it was - years later. She even compared her relationship to mine, which was exceedingly abusive, and said I was "lucky" because my ex had moved out of town and hers, who was not abusive, had not. Her breakup was

In [367]:
y_pred = classifier.predict(x_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

ValueError: X has 36527 features, but RandomForestClassifier is expecting 32090 features as input.